# Actor model

> Takes in the current observation and the current latent, and outputs the next action distribution.

In [ ]:
#| default_exp models.actor

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export 
import numpy as np
import torch
import torch.distributions as td
import torch.nn as nn
from fastcore.utils import *
from fastcore.all import *

In [ ]:
#| export
class DiscreteActionModel(nn.Module):
    def __init__(
        self,
        action_size,
        deter_size,
        stoch_size,
        embedding_size,
        actor_info,
        expl_info
    ):
        super().__init__()
        self.action_size = action_size
        self.deter_size = deter_size
        self.stoch_size = stoch_size
        self.embedding_size = embedding_size
        self.layers = actor_info['layers']
        self.node_size = actor_info['node_size']
        self.act_fn = actor_info['activation']
        self.dist = actor_info['dist']
        self.act_fn = actor_info['activation']
        self.train_noise = expl_info['train_noise']
        self.eval_noise = expl_info['eval_noise']
        self.expl_min = expl_info['expl_min']
        self.expl_decay = expl_info['expl_decay']
        self.expl_type = expl_info['expl_type']
        self.model = self._build_model()


In [ ]:
#| export
@patch
def _build_model(self: DiscreteActionModel):
    model = [nn.Linear(self.deter_size + self.stoch_size, self.node_size)]
    model += [self.act_fn()]
    for i in range(1, self.layers):
        model += [nn.Linear(self.node_size, self.node_size)]
        model += [self.act_fn()]

    if self.dist == 'one_hot':
        model += [nn.Linear(self.node_size, self.action_size)]
    else:
        raise NotImplementedError
    return nn.Sequential(*model) 

In [ ]:
#| export 
@patch
def forward(self: DiscreteActionModel, model_state):
    action_dist = self.get_action_dist(model_state)
    action = action_dist.sample()
    action = action + action_dist.probs - action_dist.probs.detach()
    return action, action_dist

In [ ]:
#| export 
@patch
def get_action_dist(self: DiscreteActionModel, modelstate):
    logits = self.model(modelstate)
    if self.dist == 'one_hot':
        return torch.distributions.OneHotCategorical(logits=logits)         
    else:
        raise NotImplementedError
        

In [ ]:
#| export
@patch
def add_exploration(self: DiscreteActionModel, action: torch.Tensor, itr: int, mode='train'):
    if mode == 'train':
        expl_amount = self.train_noise
        expl_amount = expl_amount - itr/self.expl_decay
        expl_amount = max(self.expl_min, expl_amount)
    elif mode == 'eval':
        expl_amount = self.eval_noise
    else:
        raise NotImplementedError
        
    if self.expl_type == 'epsilon_greedy':
        if np.random.uniform(0, 1) < expl_amount:
            index = torch.randint(0, self.action_size, action.shape[:-1], device=action.device)
            action = torch.zeros_like(action)
            action[:, index] = 1
        return action

    raise NotImplementedError

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()